In [1]:
from torchvision import transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import MNIST

In [2]:
class FashionMNIST2D(MNIST):
    '''Implement Dataset with FashionMnist'''
    urls = [
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz',
        'http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz',
    ]

class FashionModel(nn.Module):
    def __init__(self):
        super(FashionModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)

        self.conv2_drop = nn.Dropout2d()

        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))

        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [3]:
FASHION_DIR = '/home/erikreppel/data/fashion-mnist/'
MNIST_DIR = '/home/erikreppel/data/mnist/'

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))])

trainset = FashionMNIST2D(FASHION_DIR, download=True,
            transform=transform)
testset = FashionMNIST2D(FASHION_DIR, train=False, download=True,
                                    transform=transform)

batch_size = 256

train_loader = torch.utils.data.DataLoader(dataset=trainset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=testset,
                                          batch_size=batch_size,
                                          shuffle=True)

In [4]:
from experiment import Experiment

model = FashionModel()

exp = Experiment(model, viz_logging=True)

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

exp.compile(optimizer, criterion)

In [6]:
exp.fit(train_loader, n_epoch=10, valid_loader=test_loader, valid_freq=1)

TRAIN epoch: 0 avg_loss: 0.004187234059969584
VALIDATION: Epoch: 0 
Avg loss: 0.0024, Accuracy: 0.7623
VALIDATION: Epoch: 1 
Avg loss: 0.0021, Accuracy: 0.7933
TRAIN epoch: 2 avg_loss: 0.002485749518374602
VALIDATION: Epoch: 2 
Avg loss: 0.0020, Accuracy: 0.8074
VALIDATION: Epoch: 3 
Avg loss: 0.0018, Accuracy: 0.8188
TRAIN epoch: 4 avg_loss: 0.0022139920214811963
VALIDATION: Epoch: 4 
Avg loss: 0.0018, Accuracy: 0.8318
VALIDATION: Epoch: 5 
Avg loss: 0.0017, Accuracy: 0.8404
TRAIN epoch: 6 avg_loss: 0.00205288261671861
VALIDATION: Epoch: 6 
Avg loss: 0.0016, Accuracy: 0.8515
VALIDATION: Epoch: 7 
Avg loss: 0.0016, Accuracy: 0.8543
TRAIN epoch: 8 avg_loss: 0.0019563588003317514
VALIDATION: Epoch: 8 
Avg loss: 0.0015, Accuracy: 0.8619
VALIDATION: Epoch: 9 
Avg loss: 0.0015, Accuracy: 0.8659


In [7]:
exp.evaluate(test_loader)

TEST: Avg loss: 0.0015, Accuracy: 0.8659
